# 单因素ANOVA

In [31]:
import pandas as pd
from statsmodels.formula.api import ols
import statsmodels.api as sm

data = pd.read_csv("data_aggregated.csv", encoding="utf-8")
data.head()

,Index,Nov-24,Oct-24,Sep-24,Aug-24,Jul-24,Jun-24,May-24,Apr-24,Mar-24,...,Mar-19,Nov-18,Oct-18,Sep-18,Aug-18,Jul-18,Jun-18,May-18,Apr-18,Mar-18
0,Online,16675.200000,14704.100000,12575.500000,12568.400000,12792.600000,13322.600000,13558.500000,11028.50000,11546.400000,...,8395.900000,10149.500000,7754.500000,7589.100000,7332.800000,7052.700000,8119.400000,6899.100000,6473.000000,7047.20000
1,Total,43763.000000,45396.400000,41112.300000,38725.800000,37757.400000,40731.600000,39211.000000,35699.10000,39019.900000,...,31725.700000,35259.700000,35534.400000,32005.400000,31542.300000,30733.700000,30841.600000,30359.100000,28541.900000,29193.60000
2,Offline,27087.800000,30692.300000,28536.800000,26157.400000,24964.800000,27409.000000,25652.500000,24670.60000,27473.500000,...,23329.800000,25110.200000,27779.900000,24416.300000,24209.500000,23681.000000,22722.200000,23460.000000,22068.900000,22146.40000
3,Online/Offline,0.615598,0.479081,0.440677,0.480491,0.512425,0.486067,0.528545,0.44703,0.420274,...,0.359879,0.404198,0.279141,0.310821,0.302889,0.297821,0.357333,0.294079,0.293309,0.31821


In [32]:
data_filtered = data.set_index("Index").T.reset_index()
data_filtered.columns = ["Month-Year"] + list(data_filtered.columns[1:])  

data_filtered["Year"] = data_filtered["Month-Year"].str.extract(r"(\d{2,4})").astype(int)

long_data = data_filtered.melt(id_vars=["Year"], value_vars=["Online/Offline"], var_name="Metric", value_name="Ratio")

long_data["Ratio"] = pd.to_numeric(long_data["Ratio"], errors="coerce")

long_data = long_data.dropna()


print("\n处理后的数据:")
long_data



处理后的数据:


,Year,Metric,Ratio
0,24,Online/Offline,0.615598
1,24,Online/Offline,0.479081
2,24,Online/Offline,0.440677
3,24,Online/Offline,0.480491
4,24,Online/Offline,0.512425
...,...,...,...
58,18,Online/Offline,0.297821
59,18,Online/Offline,0.357333
60,18,Online/Offline,0.294079
61,18,Online/Offline,0.293309


In [33]:
# 单因素 ANOVA 分析
model = ols("Ratio ~ C(Year)", data=long_data).fit()  
anova_table = sm.stats.anova_lm(model, typ=2)  
print("\nANOVA 结果：")
print(anova_table)

# 判断显著性
p_value = anova_table["PR(>F)"][0] 
if p_value < 0.05:
    print("\n结果显著(拒绝零假设)：不同年份之间电商销售额和传统零售销售额的比值存在显著差异。")
else:
    print("\n结果不显著(无法拒绝零假设)：不同年份之间电商销售额和传统零售销售额的比值没有显著差异。")


ANOVA 结果：
            sum_sq    df         F        PR(>F)
C(Year)   0.280605   7.0  11.40509  7.782957e-09
Residual  0.193313  55.0       NaN           NaN

结果显著(拒绝零假设)：不同年份之间电商销售额和传统零售销售额的比值存在显著差异。


# 双因素ANOVA

In [34]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

df = pd.read_csv('data_aggregated_2.csv', encoding='gbk')

print("原始数据：")
df.head()

原始数据：


,Table,2023,2022,2021,2020,2019,2018
0,全国,0.486283,0.456650,0.422287,0.428608,0.352425,0.313032
1,北京市,4.914489,4.223295,3.978082,2.418758,1.358125,1.023274
2,天津市,1.162497,1.076020,0.849945,0.951040,1.131696,0.424742
3,河北省,0.448165,0.440037,0.308073,0.274425,0.227131,0.202742
4,山西省,0.158916,0.126189,0.126700,0.112846,0.087151,0.088795


In [35]:
# 转换为长格式
df_long = pd.melt(df, id_vars=['Table'], var_name='Year', value_name='Sales_Ratio')

# 重命名列
df_long.rename(columns={'Table': 'Region'}, inplace=True)

# 查看转换后的数据
print("\n转换后的长格式数据:")
df_long.head()


转换后的长格式数据:


,Region,Year,Sales_Ratio
0,全国,2023,0.486283
1,北京市,2023,4.914489
2,天津市,2023,1.162497
3,河北省,2023,0.448165
4,山西省,2023,0.158916


In [36]:
df_long['Sales_Ratio'] = pd.to_numeric(df_long['Sales_Ratio'], errors='coerce')

df_long = df_long.dropna(subset=['Sales_Ratio'])

# 将年份聚合为周期（18-19、20-21、22-23）
def map_to_period(year):
    if year in ['2018', '2019']:
        return '2018-2019'
    elif year in ['2020', '2021']:
        return '2020-2021'
    elif year in ['2022', '2023']:
        return '2022-2023'
    else:
        return '其他'

df_long['Period'] = df_long['Year'].apply(map_to_period)

print("\n添加周期后的数据：")
df_long.head()


添加周期后的数据：


,Region,Year,Sales_Ratio,Period
0,全国,2023,0.486283,2022-2023
1,北京市,2023,4.914489,2022-2023
2,天津市,2023,1.162497,2022-2023
3,河北省,2023,0.448165,2022-2023
4,山西省,2023,0.158916,2022-2023


In [37]:
df_long['Period'] = df_long['Period'].astype('category')
df_long['Region'] = df_long['Region'].astype('category')

# 构建双因素 ANOVA 模型
model = ols('Sales_Ratio ~ C(Period) * C(Region)', data=df_long).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print("\n双因素 ANOVA 结果（包含交互效应）：")
print(anova_table)

# 模型摘要
print("\n模型摘要：")
model.summary()


双因素 ANOVA 结果（包含交互效应）：
                        sum_sq    df           F        PR(>F)
C(Period)             2.296506   2.0   44.835609  1.775330e-14
C(Region)            91.188200  31.0  114.858294  5.434523e-63
C(Period):C(Region)  13.221900  62.0    8.326981  5.523955e-20
Residual              2.458588  96.0         NaN           NaN

模型摘要：


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Sales_Ratio   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.955
Method:                 Least Squares   F-statistic:                     43.86
Date:                Mon, 23 Dec 2024   Prob (F-statistic):           6.76e-53
Time:                        00:05:16   Log-Likelihood:                 145.92
No. Observations:                 192   AIC:                            -99.85
Df Residuals:                      96   BIC:                             212.9
Df Model:                          95                                         
Covariance Type:            nonrobust                                         
================================================================================================================
                                                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Intercept                                        1.6811      0.113     14.856      0.000       1.456       1.906
C(Period)[T.2020-2021]                           1.4449      0.160      9.029      0.000       1.127       1.763
C(Period)[T.2022-2023]                           0.6616      0.160      4.134      0.000       0.344       0.979
C(Region)[T.云南省]                                -1.6002      0.160     -9.999      0.000      -1.918      -1.283
C(Region)[T.全国]                                 -1.3484      0.160     -8.426      0.000      -1.666      -1.031
C(Region)[T.内蒙古自治区]                             -1.5919      0.160     -9.948      0.000      -1.910      -1.274
C(Region)[T.北京市]                                -0.4904      0.160     -3.064      0.003      -0.808      -0.173
C(Region)[T.吉林省]                                -1.5480      0.160     -9.673      0.000      -1.866      -1.230
C(Region)[T.四川省]                                -1.5089      0.160     -9.429      0.000      -1.827      -1.191
C(Region)[T.天津市]                                -0.9029      0.160     -5.642      0.000      -1.221      -0.585
C(Region)[T.宁夏回族自治区]                            -1.6061      0.160    -10.036      0.000      -1.924      -1.288
C(Region)[T.安徽省]                                -1.5322      0.160     -9.574      0.000      -1.850      -1.215
C(Region)[T.山东省]                                -1.5261      0.160     -9.536      0.000      -1.844      -1.208
C(Region)[T.山西省]                                -1.5931      0.160     -9.955      0.000      -1.911      -1.275
C(Region)[T.广东省]                                -0.6600      0.160     -4.124      0.000      -0.978      -0.342
C(Region)[T.广西壮族自治区]                            -1.5727      0.160     -9.827      0.000      -1.890      -1.255
C(Region)[T.新疆维吾尔自治区]                           -1.6271      0.160    -10.167      0.000      -1.945      -1.309
C(Region)[T.江苏省]                                -1.3268      0.160     -8.291      0.000      -1.644      -1.009
C(Region)[T.江西省]                                -1.5085      0.160     -9.426      0.000      -1.826      -1.191
C(Region)[T.河北省]                                -1.4662      0.160     -9.162      0.000      -1.784      -1.148
C(Region)[T.河南省]                                -1.5792      0.160     -9.868      0.000      -1.897      -1.262
C(Region)[T.浙江省]                                -0.2740      0.160     -1.712      0.090      -0.592       0.044
C(Region)[T.海南省]                                -1.4463      0.160     -9.038      0.000      -1.764      -1.129
C(Region)[T.湖北省]                                -1.5390      0.160     -9.617      0.000      -1.857      -1.221
C(Region)[T.湖南省]                

In [38]:
# 多重比较测试（Tukey HSD）
if (anova_table['PR(>F)'] < 0.05).any():
    # Tukey HSD 测试 - Period
    tukey_period = pairwise_tukeyhsd(endog=df_long['Sales_Ratio'],
                                     groups=df_long['Period'],
                                     alpha=0.05)
    print("\nTukey HSD 测试 - Period：")
    print(tukey_period)
    
    # Tukey HSD 测试 - Region
    tukey_region = pairwise_tukeyhsd(endog=df_long['Sales_Ratio'],
                                     groups=df_long['Region'],
                                     alpha=0.05)
    print("\nTukey HSD 测试 - Region：")
    print(tukey_region)
else:
    print("\nANOVA 分析未显示显著性差异，无需进行 Tukey HSD 测试。")


Tukey HSD 测试 - Period：
   Multiple Comparison of Means - Tukey HSD, FWER=0.05   
  group1    group2  meandiff p-adj   lower  upper  reject
---------------------------------------------------------
2018-2019 2020-2021   0.1906 0.3258 -0.1235 0.5046  False
2018-2019 2022-2023   0.2583 0.1295 -0.0557 0.5724  False
2020-2021 2022-2023   0.0678 0.8666 -0.2462 0.3818  False
---------------------------------------------------------

Tukey HSD 测试 - Region：
  Multiple Comparison of Means - Tukey HSD, FWER=0.05   
 group1   group2  meandiff p-adj   lower   upper  reject
--------------------------------------------------------
     上海市      云南省  -2.2819    0.0 -3.0274 -1.5365   True
     上海市       全国  -1.9734    0.0 -2.7188 -1.2279   True
     上海市   内蒙古自治区  -2.2767    0.0 -3.0222 -1.5313   True
     上海市      北京市   0.6027 0.3419 -0.1427  1.3482  False
     上海市      吉林省  -2.2228    0.0 -2.9683 -1.4774   True
     上海市      四川省  -2.1848    0.0 -2.9303 -1.4394   True
     上海市      天津市  -1.4506    0.0